In [1]:
from axelrod import Agent, Axelrod
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.stats import pearsonr
import random as rd
from inequality import gini

In [13]:
model = Axelrod()
model.run_sim()

Initial state of the model:

[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------
[1][1][1][1][1][1][1]
-----------------------------------

Running simulation...
Final state of the model:
[-0.010769021623065422][2.384117796365886][2.169111199749887][-0.004227121653379304][4.515710205536783][0.8024129420131492][0.035467122127002614]
-----------------------------------
[3.7105412419452413][-0.006314391328174168][1.2232783809269658][0.2057182372574144][1.012856397120696][0.21602743325719445][-0.007325382324823807]
-----------------------------------
[0.7003318615325901][0.8160819656503698][0.0432716985913202][2.5142273261813313][-0.010430648721324863][-0.00502234695455629][4.798713103021019]
------

In [14]:
def plot_money_evolve(model):
    plt.figure(figsize=(3, 2), dpi=200)
    for i in range (5**2):
        if model.agents[i].greediness > 0.5:
            plt.plot (model.results[i], 'r')
        else:
            plt.plot (model.results[i], 'b')
#    plt.ax.set_yscale('log')
    plt.title('0.5% taxes/year')
    plt.xlabel('time')
    plt.ylabel('geld')
    plt.show()
plot_money_evolve(model)

2023-03-31 19:33:50.656 python[91694:3341336] +[CATransaction synchronize] called within transaction


In [15]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

# assume your model.results dictionary contains N vectors, each of dimension D
N = len(model.results)
D = len(model.results[0])  # assuming all vectors have the same dimension

# initialize a NxD matrix with zeros
matrix = np.zeros((N, D))

# fill in the matrix with the vectors from the dictionary
for i in range(N):
    matrix[i] = model.results[i]
    

# assume your matrix is called 'matrix'
n_features, n_timepoints = matrix.shape

# create an empty array to store the Gini Index for each timepoint
gini_index = np.zeros(n_timepoints)

# loop over each timepoint
for t in range(n_timepoints):
    # get the vector at this timepoint
    vector = matrix[:,t]
    '''
    # calculate the highest 10% and lowest 50% of values in the vector
    sorted_vector = np.sort(vector)
    top_10_percent = sorted_vector[-int(len(sorted_vector) * 0.1):]
    bottom_50_percent = sorted_vector[:int(len(sorted_vector) * 0.5)]
    
    # calculate the average of these values
    avg_top_10_percent = np.mean(top_10_percent)
    avg_bottom_50_percent = np.mean(bottom_50_percent)
    
    # calculate the Gini Index for this timepoint
    gini_index[t] = avg_top_10_percent - avg_bottom_50_percent
    '''
    gini_index[t] = gini(vector)


In [17]:
# plot the Gini Index over time
plt.plot(gini_index)
plt.title('gini coefficient over time: 0.5% taxes')
plt.xlabel('Time')
plt.ylabel('Gini Index')
plt.show()

2023-03-31 19:36:15.194 python[91694:3341336] +[CATransaction synchronize] called within transaction
2023-03-31 19:36:22.867 python[91694:3341336] +[CATransaction synchronize] called within transaction


# not yet complete!

In [43]:
# takes a 1D vector and gives the order of all elements that are bigger than 'lower_bound'. 
# the order of all values lower than 'lower_bound' is set to 0

def get_order (vector, lower_bound):
    vector = [0 if x < lower_bound else x for x in vector]
    sorted_indices = sorted(range(len(vector)), key=lambda i: vector[i])
    ordered = [0] * len(vector)
    for rank, index in enumerate(sorted_indices):
        ordered[index] = rank+1
    ordered = [0 if x == 0 else y for x, y in zip(vector, ordered)]
    return (ordered)
get_order([1,2,3,4,5,6,7,8,9], 3)

[0, 0, 3, 4, 5, 6, 7, 8, 9]

In [67]:
# assume your model.results dictionary contains N vectors, each of dimension D
N = len(model.results)
D = len(model.results[0])  # assuming all vectors have the same dimension

# initialize a NxD matrix with zeros
matrix = np.zeros((N, D))

# fill in the matrix with the vectors from the dictionary
for i in range(N):
    matrix[i] = model.results[i]

In [59]:
# definiere wie viel änderung dort ist!
vector1 = [0, 6, 2, 9, 1, 3]

# create the order vector for vector1
sorted_indices = sorted(range(len(vector1)), key=lambda i: vector1[i])
order_vector1 = [0] * len(vector1)
for rank, index in enumerate(sorted_indices):
    order_vector1[index] = rank + 1

print(order_vector1)
# output: [1, 4, 3, 5, 2]

[1, 5, 3, 6, 2, 4]


In [44]:
def show_histogram (model, state):
    distribution=[]
    for i in range (5**2):
        distribution.append(model.results[i][state])
    plt.hist(distribution)
    plt.title('money distribution')
    plt.xlabel('money')
    plt.ylabel('agents')
    plt.show()

In [45]:
# compute the greediness and delta for our simulation
delta = []
greediness = []
for i in range(5**2):
    delta.append(model.results[i][0]-model.results[i][-1])
    greediness.append(model.agents[i].greediness)


### compute and visualise correlations

In [46]:
plt.figure(figsize=(3, 2), dpi=200)
# create a dataframe with the two vectors
data = {'greediness': greediness, 'delta': np.abs(delta)}
df = pd.DataFrame(data)

# plot the scatter plot with regression line and 95% confidence interval
sns.regplot(x='greediness', y='delta', data=df, ci=95)

# add a title and axis labels
plt.title(r'correlation between $likelihood to play$ and $|\Delta|$')
plt.xlabel('likelihood to play')
plt.ylabel('$|\Delta|$')

plt.show()

In [139]:
# calculate the correlation and p-value
corr, p_value = pearsonr(greediness, np.abs(delta))

# print the results
print("Correlation: ", corr)
print("P-value: ", p_value)

Correlation:  0.48726075450727097
P-value:  0.01349353826946781


### implementation ideas

- delta money (veränderung) abhängig von greed 
- Varianz (SD von den 25 Geld werten zu zeit t) / Zeit 


- Einkommensverteilung 
- Graph mit Einkommensverteilung 
- correlation between 
    - reich & greedy
    - arm.. 

